In [1]:
import os
import sys
import time
from enum import Enum

import matplotlib.pyplot as plt
import pandas as pd
import requests
from functional import seq

# Load the "autoreload" extension
% reload_ext autoreload

# always reload modules marked with "%aimport"
% autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, os.pardir, 'src')
sys.path.append(src_dir)

In [2]:
class PeriodColumns(Enum):
    ID = 'id'
    OPEN = 'open'
    CLOSE = 'close'
    HIGH = 'high'
    LOW = 'low'
    VOLUME = 'volume'
    WEIGHTED_AVERAGE = 'weightedAverage'
    BASE_PRICE = 'basePrice'
    BASE_PRICE_BIN = 'basePriceBin'
    
    def __init__(self, column_name):
        self.column_name = column_name


class BinColumns(Enum):
    BIN_ID = 'binId'
    AGG_VOLUME = 'aggVolume'
    LAST_PREVIOUS_BIN_ID = 'recentStrongestBinId'
    CONTAINS_BUY_ORDER = 'containsBuyOrder'

    OLD_LAST_AGG_VOLUME = 'oldLastAggVolume'
    OLD_LAST_MAX_VOLUME = 'oldLastMaxVolume'
    OLD_LAST_BASE_PRICE = 'oldLastBasePrice'
    OLD_ONE_BEFORE_LAST_AGG_VOLUME = 'oldOneBeforeLastAggVolume'
    OLD_ONE_BEFORE_LAST_MAX_VOLUME = 'oldOneBeforeLastMaxVolume'
    OLD_PERIOD_START = 'oldPeriodStart'
    OLD_PERIOD_END = 'oldPeriodEnd'
    OLD_BASE_SCORE = 'oldBaseScore'

    NEW_LAST_AGG_VOLUME = 'newLastAggVolume'
    NEW_LAST_MAX_VOLUME = 'newLastMaxVolume'
    NEW_LAST_BASE_PRICE = 'newLastBasePrice'
    NEW_ONE_BEFORE_LAST_AGG_VOLUME = 'newOneBeforeLastAggVolume'
    NEW_ONE_BEFORE_LAST_MAX_VOLUME = 'newOneBeforeLastMaxVolume'
    NEW_PERIOD_START = 'newPeriodStart'
    NEW_PERIOD_END = 'newPeriodEnd'
    NEW_BASE_SCORE = 'newBaseScore'
    
    def __init__(self, column_name):
        self.column_name = column_name


def calculate_profit_in_percents(sell_price, buy_price):
    return (sell_price / buy_price - 1 - trading_fee_decimal) * 100


def add_trading_fee_to_volume(volume):
    return volume / (1 + trading_fee_decimal)


def subtract_trading_fee_from_volume(volume):
    return volume / (1 - trading_fee_decimal)


def calculate_trading_fee(price, volume):
    return price * volume * trading_fee_decimal


class Wallet:
    def __init__(self, base_currency_symbol, target_currency_symbol):
        self.base_currency_symbol = base_currency_symbol
        self.base_currency_amount = 0
        self.base_currency_paid_on_fees = 0
        self.target_currency_symbol = target_currency_symbol
        self.target_currency_amount = 0

    def buy(self, price, volume):
        volume_with_fee = add_trading_fee_to_volume(volume)
        
        if self.base_currency_amount < price * volume_with_fee:
            bought_volume_without_fee = subtract_trading_fee_from_volume(self.base_currency_amount / price)
            self.target_currency_amount += bought_volume_without_fee
            self.base_currency_amount = 0
            self.base_currency_paid_on_fees += calculate_trading_fee(price, bought_volume_without_fee)
            return bought_volume_without_fee
        else:
            self.target_currency_amount += volume
            self.base_currency_amount -= price * volume_with_fee
            self.base_currency_paid_on_fees += calculate_trading_fee(price, volume)
            return volume
        
    def sell(self, price, volume):
        volume_with_fee = add_trading_fee_to_volume(volume)
        
        sold_volume_with_fee = min(self.target_currency_amount, volume_with_fee)
        sold_volume_without_fee = subtract_trading_fee_from_volume(sold_volume_with_fee)
        self.target_currency_amount -= sold_volume_with_fee
        self.base_currency_amount += price * sold_volume_without_fee
        self.base_currency_paid_on_fees += calculate_trading_fee(price, sold_volume_without_fee)
        return sold_volume_without_fee

    def get_total_in_base_currency(self, current_price):
        return self.base_currency_amount + current_price * self.target_currency_amount

    def to_string(self, current_price):
        return '\nWALLET: {} {:7.2f}, {} {:7.2f} => in total {} {:7.2f}'.format(
            self.base_currency_symbol, self.base_currency_amount,
            self.target_currency_symbol, self.target_currency_amount,
            self.base_currency_symbol, self.get_total_in_base_currency(current_price))


# Represents an outstanding order.
class OutstandingOrder:
    def __init__(self, price, volume):
        self.price = price
        self.volume = volume

    def to_string(self, current_price):
        return self.__str__()

    def draw(self, plt):
        pass


# Only drawing of sell order is currently supported. The reason is that buy orders are 
# immediately after their creation executed.
class OutstandingBuyOrder(OutstandingOrder):
    def __init__(self, buy_price, buy_volume, base_price, created_on_period_index, created_on_bin_index):
        super().__init__(buy_price, buy_volume)
        self.base_price = base_price
        self.created_on_period_index = created_on_period_index
        self.created_on_bin_index = created_on_bin_index

    def calculate_potential_profit_in_percents(self):
        return calculate_profit_in_percents(self.base_price, self.price)

    def __str__(self):
        return '{:4}: {} ({} {:7.2f}, {} {:7.2f}), {} {:9.5f}, {} {:6.0f}, {} {:6.2f}%'.format(
            'BUY',
            'prices', 'base', self.base_price, 'buy', self.price,
            'buyVol', self.volume,
            'createdOn', self.created_on_period_index,
            'potentialProfit', self.calculate_potential_profit_in_percents())


class OutstandingSellOrder(OutstandingOrder):
    def __init__(self, sell_price, buy_order_execution):
        super().__init__(sell_price, buy_order_execution.volume)
        self.buy_order_execution = buy_order_execution

    def calculate_planned_profit_in_percents(self):
        return calculate_profit_in_percents(self.price, self.buy_order_execution.price)

    def calculate_current_profit_in_percents(self, current_price):
        return calculate_profit_in_percents(current_price, self.buy_order_execution.price)

    def to_string(self, current_price):
        return '{:4}: {} ({} {:7.2f}, {} {:7.2f}, {} {:7.2f}), {} {:9.5f}, {} {:6.0f}, {} ({} {:6.2f}%, {} {:6.2f}%)'.format(
            'SELL',
            'prices', 'base', self.buy_order_execution.order.base_price, 'bought', self.buy_order_execution.price,
            'sell', self.price,
            'sellVol', self.volume,
            'buyCreatedOn', self.buy_order_execution.order.created_on_period_index,
            'profit', 'planned', self.calculate_planned_profit_in_percents(), 'current',
            self.calculate_current_profit_in_percents(current_price))

    def draw(self, plt):
        color = 'red'
        line_style = 'dashed'
        line_width = (self.calculate_planned_profit_in_percents() / 10).astype('int') + 1

        # Vertical line from the buy point
        plt.plot([self.buy_order_execution.order.created_on_period_index,
                  self.buy_order_execution.order.created_on_period_index],
                 [self.buy_order_execution.price, self.price],
                 linestyle=line_style, color=color, linewidth=line_width)
        # Horizontal line to the sell point (different than in TradingHistoryEntry class in order to
        # mitigate overlapping)
        # TODO does not work nicely: plt.plot([self.created_on_period_index, plt.gca().get_xlim()[1]],
        plt.plot([self.buy_order_execution.order.created_on_period_index,
                  self.buy_order_execution.order.created_on_period_index + 500],
                 [self.price, self.price],
                 linestyle=line_style, color=color, linewidth=line_width)


class OutstandingOrderManager:
    def __init__(self):
        self.orders = []

    def to_string(self, current_price):
        concatenated_orders = '\n'.join(order.to_string(current_price) for order in self.orders)
        return 'ORDERS:\n{}'.format(concatenated_orders)


class OrderExecution:
    def __init__(self, order, price, volume, executed_on_period_index):
        self.order = order
        self.price = price
        self.volume = volume
        self.executed_on_period_index = executed_on_period_index


class BuyOrderExecution(OrderExecution):
    def __init__(self, buy_order, bought_price, bought_volume, bought_on_period_index):
        super().__init__(buy_order, bought_price, bought_volume, bought_on_period_index)

    def calculate_potential_profit_in_percents(self):
        return calculate_profit_in_percents(self.order.base_price, self.price)

    def __str__(self):
        return '{:6}: {} ({} {:7.2f}, {} {:7.2f}), {} {:9.5f}, {} {:6.0f}, {} {:6.2f}%'.format(
            'BOUGHT',
            'prices', 'base', self.order.base_price, 'bought', self.price,
            'boughtVol', self.volume,
            'boughtOn', self.order.executed_on_period_index,
            'potentialProfit', self.calculate_potential_profit_in_percents())

    def draw(self, plt):
        color = 'green'

        plt.plot(self.executed_on_period_index, self.price, marker='o', color=color)

        line_style = 'dotted'
        plt.plot([self.executed_on_period_index, self.executed_on_period_index],
                 [self.price, self.order.base_price],
                 linestyle=line_style, color=color)


class SellOrderExecution(OrderExecution):
    def __init__(self, sell_order, sold_price, sold_volume, sold_on_period_index):
        super().__init__(sell_order, sold_price, sold_volume, sold_on_period_index)

    def calculate_realized_profit_in_percents(self):
        return calculate_profit_in_percents(self.price, self.order.buy_order_execution.price)

    def __str__(self):
        return '{:6}: {} ({} {:7.2f}, {} {:7.2f}, {} {:7.2f}), {} {:9.5f}, {} {:6.0f}, {} {:6.2f}%'.format(
            'SOLD',
            'prices', 'base', self.order.buy_order_execution.order.base_price, 'bought',
            self.order.buy_order_execution.price, 'sold', self.price,
            'soldVol', self.volume,
            'soldOn', self.executed_on_period_index,
            'profit', self.calculate_realized_profit_in_percents())

    def draw(self, plt):
        color = 'red'

        plt.plot(self.order.executed_on_period_index, self.order.price, marker='o', color=color)

        line_style = 'dotted'
        # Horizontal line from the buy point
        plt.plot([self.order.buy_order_execution.executed_on_period_index, self.executed_on_period_index],
                 [self.order.buy_order_execution.price, self.order.buy_order_execution.price],
                 linestyle=line_style, color=color)
        # Vertical line to the sell point
        plt.plot([self.executed_on_period_index, self.executed_on_period_index],
                 [self.order.buy_order_execution.price, self.price],
                 linestyle=line_style, color=color)


class TradingHistory:
    def __init__(self):
        self.order_executions = []

    def __str__(self):
        concatenated_order_executions = '\n'.join(order_execution for order_execution in self.order_executions if
                                                  isinstance(order_execution, SellOrderExecution))
        return 'TRADING_HISTORY:\n{}'.format(concatenated_order_executions)


class ExchangePlatform:
    def __init__(self, base_currency_symbol, target_currency_symbol):
        self.wallet = Wallet(base_currency_symbol, target_currency_symbol)
        self.outstanding_order_manager = OutstandingOrderManager()
        self.trading_history = TradingHistory()

    def deposit(self, base_currency_amount):
        self.wallet.base_currency_amount += base_currency_amount

    def determine_buy_order_volume(self, potential_profit_in_percents, potential_profit_lower_bound_in_percents,
                                   last_base_score, current_price):

        # 1% of the total money in the wallet expressed in the target currency
        volume_unit_of_target_currency_to_buy = (self.wallet.get_total_in_base_currency(
            current_price) / 100) / current_price

        profit_coefficient = (potential_profit_in_percents - potential_profit_lower_bound_in_percents) * last_base_score

        return min(profit_coefficient, 5) * volume_unit_of_target_currency_to_buy

    def create_buy_order_if_wanted(self, current_period_index, current_price, last_base_score,
                                   last_base_last_base_price, current_bin_index, price_bins):

        if price_bins.loc[current_bin_index, BinColumns.CONTAINS_BUY_ORDER.value] == 1:
            # Only one not resolved buy order per bin.
            return

        potential_profit_in_percents = calculate_profit_in_percents(last_base_last_base_price, current_price)
        potential_profit_lower_bound_in_percents = 3
        if potential_profit_in_percents > potential_profit_lower_bound_in_percents:
            volume = self.determine_buy_order_volume(potential_profit_in_percents,
                                                     potential_profit_lower_bound_in_percents, last_base_score,
                                                     current_price)

            new_order = OutstandingBuyOrder(current_price, volume, last_base_last_base_price, current_period_index,
                                            current_bin_index)
            self.outstanding_order_manager.orders.append(new_order)

            price_bins.set_value(current_bin_index, BinColumns.CONTAINS_BUY_ORDER.value, 1)

    def execute_buy_order(self, buy_order, current_price, current_period_index, price_bins):
        self.outstanding_order_manager.orders.remove(buy_order)
        if buy_order.price < current_price:
            # The price is higher than in buy order -> too expensive to buy. The order is being removed.
            price_bins.set_value(buy_order.created_on_bin_index, BinColumns.CONTAINS_BUY_ORDER.value, 0)

        else:
            realized_volume = self.wallet.buy(current_price, buy_order.volume)

            buy_order_execution = BuyOrderExecution(buy_order, current_price, realized_volume, current_period_index)
            self.trading_history.order_executions.append(buy_order_execution)

            # PLAN sell price - needs to be improved
            sell_price = buy_order.base_price

            new_sell_order = OutstandingSellOrder(sell_price, buy_order_execution)
            self.outstanding_order_manager.orders.append(new_sell_order)

    def execute_sell_order_if_due(self, sell_order, current_price, current_period_index, price_bins):
        if sell_order.price > current_price:
            # The price is lower than in sell order -> too cheap to sell. The order is not resolved.
            return

        self.outstanding_order_manager.orders.remove(sell_order)
        realized_volume = self.wallet.sell(current_price, sell_order.volume)

        sell_order_execution = SellOrderExecution(sell_order, current_price, realized_volume, current_period_index)
        self.trading_history.order_executions.append(sell_order_execution)
        # TODO missing information about bin of the buy order
        price_bins.set_value(sell_order.buy_order_execution.order.created_on_bin_index,
                             BinColumns.CONTAINS_BUY_ORDER.value, 0)

    def execute_order_if_due(self, order, current_price, current_period_index, price_bins):
        if isinstance(order, OutstandingBuyOrder):
            self.execute_buy_order(order, current_price, current_period_index, price_bins)
        else:
            self.execute_sell_order_if_due(order, current_price, current_period_index, price_bins)

    def execute_due_orders(self, current_price, current_period_index, price_bins):
        [self.execute_order_if_due(order, current_price, current_period_index, price_bins) for order
         in self.outstanding_order_manager.orders]

    def to_string(self, current_price):
        return '{}\n{}\n{}'.format(self.trading_history, self.outstanding_order_manager.to_string(current_price),
                                   self.wallet.to_string(current_price))

In [3]:
def fetch_prices_json(currency_pair, now, oldest_period_offset, period_count, period_size_in_sec):
    start_date_time = now - oldest_period_offset * period_size_in_sec
    end_date_time = start_date_time + period_count * period_size_in_sec
    response = requests.get('https://poloniex.com/public?command=returnChartData', params={
        'currencyPair': currency_pair,
        'start': start_date_time,
        'end': end_date_time,
        'period': period_size_in_sec
    })
    
    print(response.url)
    
    return response.json()

In [4]:
def calculate_local_max(periods, index, column_name):
    if (0 < index < period_count - 1
            and periods.loc[index, column_name] >= periods.loc[index - 1, column_name]
            and periods.loc[index, column_name] >= periods.loc[index + 1, column_name]):
        return 1
    else:
        return 0


In [5]:
def determine_period_base_price(period_index, periods):
    if (periods.loc[period_index, PeriodColumns.CLOSE.value] >=
            periods.loc[period_index, PeriodColumns.OPEN.value]):
        return periods.loc[period_index, PeriodColumns.LOW.value]
    else:
        return periods.loc[period_index, PeriodColumns.HIGH.value]

In [6]:
# id_offset in order to render new periods after old ones.
def init_periods(periods_json, id_offset=0):
    periods = seq(periods_json).to_pandas()
    
    periods_count = len(periods)

    # To be able to find for rows by id.
    periods[PeriodColumns.ID.value] = [i + id_offset for i in range(periods_count)]
    periods.set_index(PeriodColumns.ID.value, inplace=True)
    
    periods[PeriodColumns.BASE_PRICE.value] = [determine_period_base_price(i + id_offset, periods)
                                               for i in range(periods_count)]
    
    return periods

In [7]:
def calculate_base_price_bin(period_base_price, old_price_bin_count, old_max_price):
    return (period_base_price / old_max_price * old_price_bin_count).astype('int') - 1

In [8]:
def init_old_periods(old_periods_json, max_old_price_bin_count):
    old_periods = init_periods(old_periods_json)

    old_min_price = old_periods[PeriodColumns.LOW.value].min()
    old_max_price = old_periods[PeriodColumns.HIGH.value].max()
    # The number of bins is based on the number of covered percentages.
    # If max_old_price_bin_count == 100 & old_min_price == 2000 USD & old_max_price == 5000 USD,
    # then covered 60% -> 60 bins
    old_price_bin_count = ((1 - old_min_price / old_max_price) * max_old_price_bin_count).astype('int')
    print('{}={}'.format('old_price_bin_count', old_price_bin_count))
    old_periods[PeriodColumns.BASE_PRICE_BIN.value] = (
        old_periods[PeriodColumns.BASE_PRICE.value] / old_max_price * old_price_bin_count).astype('int') - 1
    
    return old_periods, old_price_bin_count, old_max_price

In [9]:
def init_price_bins(price_bin_count):
    price_bins = pd.DataFrame({
        BinColumns.BIN_ID.value: range(price_bin_count)
    })
    price_bins.set_index(BinColumns.BIN_ID.value, inplace=True)
    price_bins[BinColumns.AGG_VOLUME.value] = [0 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_LAST_AGG_VOLUME.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_LAST_MAX_VOLUME.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_LAST_BASE_PRICE.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_ONE_BEFORE_LAST_AGG_VOLUME.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_ONE_BEFORE_LAST_MAX_VOLUME.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_PERIOD_START.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_PERIOD_END.value] = [-10000 for i in range(price_bin_count)]
    price_bins[BinColumns.OLD_BASE_SCORE.value] = [0 for i in range(price_bin_count)]
    
    return price_bins

In [10]:
def calculate_base_score(bin_last_max_volume, bin_one_before_last_max_volume, moving_overall_bin_max_volume):
    score = (bin_last_max_volume / moving_overall_bin_max_volume) * (
        bin_last_max_volume / bin_one_before_last_max_volume)  # Considers the trend.
    # Max is 2
    return min(score, 2)

In [11]:
def update_price_bin_if_bin_change(bin_index, bin_base_price,
                                   bin_agg_volume, bin_max_volume,
                                   bin_start_period_included, bin_end_period_excluded,
                                   previous_bin_index,
                                   previous_bin_agg_volume, previous_bin_max_volume,
                                   multiplication_threshold, min_volume_per_period_threshold,
                                   base_score_column, last_agg_volume_column,
                                   last_base_price_column, last_max_volume_column,
                                   one_before_last_agg_volume_column, one_before_last_max_volume_column,
                                   period_end_column, period_start_column):
    
    total_bin_agg_volume = bin_agg_volume + price_bins.loc[bin_index, BinColumns.AGG_VOLUME.value]
    price_bins.set_value(bin_index, BinColumns.AGG_VOLUME.value, total_bin_agg_volume)
    price_bins.set_value(bin_index, BinColumns.LAST_PREVIOUS_BIN_ID.value, previous_bin_index)
    
    bin_last_max_volume = price_bins.loc[bin_index, last_max_volume_column.value]
    
    # If a previous base in this price bin exists.
    if bin_last_max_volume >= 0:
        price_bins.set_value(bin_index, one_before_last_agg_volume_column.value,
                             price_bins.loc[bin_index, last_agg_volume_column.value])
        price_bins.set_value(bin_index, one_before_last_max_volume_column.value,
                             bin_last_max_volume)

    bin_mean_volume_per_period = bin_agg_volume / (bin_end_period_excluded - bin_start_period_included)

    # If a previous base in this price exists,
    # or there is a major different between the previous aggregated volume and this one.
    if (bin_last_max_volume >= 0
            or (bin_max_volume > multiplication_threshold * previous_bin_max_volume
                and bin_max_volume > multiplication_threshold * min_volume_per_period_threshold
                and bin_mean_volume_per_period > min_volume_per_period_threshold)):
        
        global moving_overall_bin_max_volume
        moving_overall_bin_max_volume = max(moving_overall_bin_max_volume, bin_max_volume)

        if bin_mean_volume_per_period > min_volume_per_period_threshold:
            price_bins.set_value(bin_index, base_score_column.value,
                                 calculate_base_score(bin_max_volume, bin_last_max_volume,
                                                      moving_overall_bin_max_volume))
        else:
            price_bins.set_value(bin_index, base_score_column.value, 0)
            
        price_bins.set_value(bin_index, last_agg_volume_column.value, bin_agg_volume)
        price_bins.set_value(bin_index, last_max_volume_column.value, bin_max_volume)
        price_bins.set_value(bin_index, last_base_price_column.value, bin_base_price)

    if (bin_max_volume > multiplication_threshold * previous_bin_max_volume
            and bin_max_volume > multiplication_threshold * min_volume_per_period_threshold
            and bin_mean_volume_per_period > min_volume_per_period_threshold):

        if price_bins.loc[bin_index, period_start_column.value] < 0:
            price_bins.set_value(bin_index, period_start_column.value, bin_start_period_included)
            
        # print('bin={:3.0f}, curr_agg_vol={:8.0f}, curr_max_vol={:8.0f}, prev_max_vol={:8.0f}'.format(
        #    bin_index, bin_agg_volume, bin_max_volume, previous_bin_max_volume))

    if price_bins.loc[bin_index, period_start_column.value] >= 0:
        price_bins.set_value(bin_index, period_end_column.value, bin_end_period_excluded)

In [12]:
def update_price_bin_olds_if_bin_change(bin_index, bin_base_price,
                                        bin_agg_volume, bin_max_volume,
                                        bin_start_period_included, bin_end_period_excluded,
                                        previous_bin_index,
                                        previous_bin_agg_volume, previous_bin_max_volume,
                                        multiplication_threshold, min_volume_per_period_threshold):
    update_price_bin_if_bin_change(bin_index, bin_base_price,
                                   bin_agg_volume, bin_max_volume,
                                   bin_start_period_included, bin_end_period_excluded,
                                   previous_bin_index,
                                   previous_bin_agg_volume, previous_bin_max_volume,
                                   multiplication_threshold, min_volume_per_period_threshold,
                                   BinColumns.OLD_BASE_SCORE, BinColumns.OLD_LAST_AGG_VOLUME,
                                   BinColumns.OLD_LAST_BASE_PRICE, BinColumns.OLD_LAST_MAX_VOLUME,
                                   BinColumns.OLD_ONE_BEFORE_LAST_AGG_VOLUME,
                                   BinColumns.OLD_ONE_BEFORE_LAST_MAX_VOLUME,
                                   BinColumns.OLD_PERIOD_END, BinColumns.OLD_PERIOD_START)

In [13]:
def update_price_bin_news_if_bin_change(bin_index, bin_base_price,
                                        bin_agg_volume, bin_max_volume,
                                        bin_start_period_included, bin_end_period_excluded,
                                        previous_bin_index,
                                        previous_bin_agg_volume, previous_bin_max_volume,
                                        multiplication_threshold, min_volume_per_period_threshold):
    update_price_bin_if_bin_change(bin_index, bin_base_price,
                                   bin_agg_volume, bin_max_volume,
                                   bin_start_period_included, bin_end_period_excluded,
                                   previous_bin_index,
                                   previous_bin_agg_volume, previous_bin_max_volume,
                                   multiplication_threshold, min_volume_per_period_threshold,
                                   BinColumns.NEW_BASE_SCORE, BinColumns.NEW_LAST_AGG_VOLUME,
                                   BinColumns.NEW_LAST_BASE_PRICE, BinColumns.NEW_LAST_MAX_VOLUME,
                                   BinColumns.NEW_ONE_BEFORE_LAST_AGG_VOLUME,
                                   BinColumns.NEW_ONE_BEFORE_LAST_MAX_VOLUME,
                                   BinColumns.NEW_PERIOD_END, BinColumns.NEW_PERIOD_START)

In [14]:
def calculate_old_bins_base_score(old_periods, multiplication_threshold, min_volume_per_period_threshold):
    
    bin_index = -10000
    bin_base_price = -10000
    bin_agg_volume = -10000
    bin_max_volume = -10000
    bin_start_period = -10000

    previous_bin_index = -10000
    previous_bin_agg_volume = -10000
    previous_bin_max_volume = -10000
    
    period_index = -10000
    
    for period_index, row in old_periods.iterrows():
        current_volume = row[PeriodColumns.VOLUME.value]
        current_bin_index = row[PeriodColumns.BASE_PRICE_BIN.value]
        
        if current_bin_index != bin_index:
            # The bin of the current period is a different one that has been being processed.
            
            if bin_index >= 0:
                update_price_bin_olds_if_bin_change(bin_index, bin_base_price,
                                                    bin_agg_volume, bin_max_volume,
                                                    bin_start_period, period_index,
                                                    previous_bin_index,
                                                    previous_bin_agg_volume, previous_bin_max_volume,
                                                    multiplication_threshold, min_volume_per_period_threshold)
                    
                previous_bin_index = bin_index
                previous_bin_agg_volume = bin_agg_volume
                previous_bin_max_volume = bin_max_volume
                
            bin_index = current_bin_index
            bin_base_price = row[PeriodColumns.BASE_PRICE.value]
            bin_agg_volume = current_volume
            bin_max_volume = current_volume
            bin_start_period = period_index
            
        else:
            bin_agg_volume += current_volume
            bin_max_volume = max(bin_max_volume, current_volume)
            if current_volume == bin_max_volume:
                bin_base_price = row[PeriodColumns.BASE_PRICE.value]
                
    if bin_index >= 0:
        # Bin from the last interaction of the loop above is being updated.
        update_price_bin_olds_if_bin_change(bin_index, bin_base_price,
                                            bin_agg_volume, bin_max_volume, bin_start_period, period_index + 1,
                                            previous_bin_index,
                                            previous_bin_agg_volume, previous_bin_max_volume,
                                            multiplication_threshold, min_volume_per_period_threshold)

In [15]:
def extract_base_relevant_values_from_bin(bin_index, price_bins):
    base_score = price_bins.loc[bin_index, BinColumns.OLD_BASE_SCORE.value]
    last_base_price = price_bins.loc[bin_index, BinColumns.OLD_LAST_BASE_PRICE.value]
    return base_score, last_base_price

In [16]:
def calculate_new_bins_base_score_and_invest(new_periods, multiplication_threshold,
                                             min_volume_per_period_threshold, exchange_platform,
                                             old_price_bin_count, old_max_price, old_periods):
    price_bins[BinColumns.CONTAINS_BUY_ORDER.value] = [0 for i in range(len(price_bins))]

    # Copies values from OLD_ columns to NEW_ columns in order to keep the context, 
    price_bins[BinColumns.NEW_LAST_AGG_VOLUME.value] = price_bins[BinColumns.OLD_LAST_AGG_VOLUME.value]
    price_bins[BinColumns.NEW_LAST_MAX_VOLUME.value] = price_bins[BinColumns.OLD_LAST_MAX_VOLUME.value]
    price_bins[BinColumns.NEW_LAST_BASE_PRICE.value] = price_bins[BinColumns.OLD_LAST_BASE_PRICE.value]
    price_bins[BinColumns.NEW_ONE_BEFORE_LAST_AGG_VOLUME.value] = price_bins[
        BinColumns.OLD_ONE_BEFORE_LAST_AGG_VOLUME.value]
    price_bins[BinColumns.NEW_ONE_BEFORE_LAST_MAX_VOLUME.value] = price_bins[
        BinColumns.OLD_ONE_BEFORE_LAST_MAX_VOLUME.value]
    price_bins[BinColumns.NEW_PERIOD_START.value] = price_bins[BinColumns.OLD_PERIOD_START.value]
    price_bins[BinColumns.NEW_PERIOD_END.value] = price_bins[BinColumns.OLD_PERIOD_END.value]
    price_bins[BinColumns.NEW_BASE_SCORE.value] = price_bins[BinColumns.OLD_BASE_SCORE.value]

    # Retrieves data about the last OLD bin.
    bin_index = old_periods[PeriodColumns.BASE_PRICE_BIN.value].iloc[-1]
    bin_base_price = price_bins.loc[bin_index, BinColumns.OLD_LAST_BASE_PRICE.value]
    bin_agg_volume = price_bins.loc[bin_index, BinColumns.OLD_LAST_AGG_VOLUME.value]
    bin_max_volume = price_bins.loc[bin_index, BinColumns.OLD_LAST_MAX_VOLUME.value]
    bin_start_period = price_bins.loc[bin_index, BinColumns.OLD_PERIOD_START.value]

    # Retrieves data about the one before last OLD bin.
    previous_bin_index = price_bins.loc[bin_index, BinColumns.LAST_PREVIOUS_BIN_ID.value]
    previous_bin_agg_volume = price_bins.loc[previous_bin_index, BinColumns.OLD_LAST_AGG_VOLUME.value]
    previous_bin_max_volume = price_bins.loc[previous_bin_index, BinColumns.OLD_LAST_MAX_VOLUME.value]

    # Information about the most recent base.
    last_base_score = -10000
    last_base_last_base_price = -10000
    if price_bins.loc[bin_index, BinColumns.OLD_BASE_SCORE.value] > 0:
        (last_base_score,
         last_base_last_base_price) = extract_base_relevant_values_from_bin(bin_index, price_bins)
    elif price_bins.loc[previous_bin_index, BinColumns.OLD_BASE_SCORE.value] > 0:
        (last_base_score,
         last_base_last_base_price) = extract_base_relevant_values_from_bin(previous_bin_index, price_bins)

    period_index = -10000
    current_weighted_average = -10000

    for period_index, row in new_periods.iterrows():
        current_volume = row[PeriodColumns.VOLUME.value]
        current_bin_index = calculate_base_price_bin(row[PeriodColumns.BASE_PRICE.value],
                                                     old_price_bin_count, old_max_price)
        current_base_price = row[PeriodColumns.BASE_PRICE.value]

        if current_bin_index != bin_index:
            # The bin of the current period is a different one that has been being processed.

            if bin_index >= 0:
                update_price_bin_news_if_bin_change(bin_index, bin_base_price,
                                                    bin_agg_volume, bin_max_volume,
                                                    bin_start_period, period_index,
                                                    previous_bin_index,
                                                    previous_bin_agg_volume, previous_bin_max_volume,
                                                    multiplication_threshold, min_volume_per_period_threshold)

                last_bin_base_score = price_bins.loc[bin_index, BinColumns.NEW_BASE_SCORE.value]
                if last_bin_base_score > 0:
                    last_base_score = last_bin_base_score
                    last_base_last_base_price = bin_base_price

                previous_bin_agg_volume = bin_agg_volume
                previous_bin_max_volume = bin_max_volume

            bin_index = current_bin_index
            bin_base_price = current_base_price
            bin_agg_volume = current_volume
            bin_max_volume = current_volume
            bin_start_period = period_index

        else:
            bin_agg_volume += current_volume
            bin_max_volume = max(bin_max_volume, current_volume)
            if current_volume == bin_max_volume:
                bin_base_price = current_base_price

        current_weighted_average = row[PeriodColumns.WEIGHTED_AVERAGE.value]

        exchange_platform.create_buy_order_if_wanted(period_index, current_weighted_average, last_base_score,
                                                     last_base_last_base_price, current_bin_index, price_bins)
        exchange_platform.execute_due_orders(current_weighted_average, period_index, price_bins)

    if bin_index >= 0:
        # Bin from the last interaction of the loop above is being updated.
        update_price_bin_news_if_bin_change(bin_index, bin_base_price,
                                            bin_agg_volume, bin_max_volume, bin_start_period, period_index + 1,
                                            previous_bin_index,
                                            previous_bin_agg_volume, previous_bin_max_volume,
                                            multiplication_threshold, min_volume_per_period_threshold)

    exchange_platform.execute_due_orders(current_weighted_average, period_index, price_bins)

    # TODO add missing bins


In [17]:
def draw_base_if_exists(price_bin, plt,
                        base_score_column, last_base_price_column, period_start_column, period_end_column,
                        drawing_period_count_offset=0):
    if price_bin[base_score_column.value] > 0:
        current_base_price = price_bin[last_base_price_column.value]
        line_width = (price_bin[base_score_column.value] / 0.3).astype('int') + 1

        # start is included, end is excluded
        plt.plot([price_bin[period_start_column.value] - drawing_period_count_offset,
                  price_bin[period_end_column.value] - 1 + drawing_period_count_offset],
                 [current_base_price, current_base_price],
                 color='purple', linewidth=line_width, label=current_base_price)


def draw_graph(old_periods, new_periods, old_new_border_period, exchange_platform):
    plt.rcParams['figure.figsize'] = (20, 10)

    plt.plot(old_periods[PeriodColumns.WEIGHTED_AVERAGE.value])
    plt.plot(new_periods[PeriodColumns.WEIGHTED_AVERAGE.value])
    plt.plot([old_new_border_period, old_new_border_period],
             plt.gca().get_ylim(),
             color='grey')

    for index, row in price_bins.iterrows():
        draw_base_if_exists(row, plt, BinColumns.OLD_BASE_SCORE, BinColumns.OLD_LAST_BASE_PRICE,
                            BinColumns.OLD_PERIOD_START, BinColumns.OLD_PERIOD_END)
        draw_base_if_exists(row, plt, BinColumns.NEW_BASE_SCORE, BinColumns.NEW_LAST_BASE_PRICE,
                            BinColumns.NEW_PERIOD_START, BinColumns.NEW_PERIOD_END)

    [order_execution.draw(plt) for order_execution in exchange_platform.trading_history.order_executions]
    [order.draw(plt) for order in exchange_platform.outstanding_order_manager.orders]

    plt.grid()
    plt.show()


In [18]:
def print_bins_result_debug_info(price_bins):
    print('\nPRICE BINS - OLD:')
    for index, row in price_bins.iterrows():
        if row[BinColumns.OLD_BASE_SCORE.value] > 0 and row[BinColumns.NEW_BASE_SCORE.value] == 0:
            print(
                '{}={:3.0f}, {}={:10.0f}, {}={:6.1f}, {}={:6.1f}, {}={:8.1f}, {}={:8.1f}, {}={:.2f}, {}={:.2f}'
                .format(
                    'bin', index,
                    'aggVol', row[BinColumns.AGG_VOLUME.value],
                    'oPrc', row[BinColumns.OLD_LAST_BASE_PRICE.value],
                    'nPrc', row[BinColumns.NEW_LAST_BASE_PRICE.value],
                    'oMaxVol', row[BinColumns.OLD_LAST_MAX_VOLUME.value],
                    'nMaxVol', row[BinColumns.NEW_LAST_MAX_VOLUME.value],
                    'oScr', row[BinColumns.OLD_BASE_SCORE.value],
                    'nScr', row[BinColumns.NEW_BASE_SCORE.value]))

    print('\nPRICE BINS - NEW:')
    for index, row in price_bins.iterrows():
        if row[BinColumns.NEW_BASE_SCORE.value] > 0:
            print(
                '{}={:3.0f}, {}={:10.0f}, {}={:6.1f}, {}={:6.1f}, {}={:8.1f}, {}={:8.1f}, {}={:.2f}, {}={:.2f}'
                .format(
                    'bin', index,
                    'aggVol', row[BinColumns.AGG_VOLUME.value],
                    'oPrc', row[BinColumns.OLD_LAST_BASE_PRICE.value],
                    'nPrc', row[BinColumns.NEW_LAST_BASE_PRICE.value],
                    'oMaxVol', row[BinColumns.OLD_LAST_MAX_VOLUME.value],
                    'nMaxVol', row[BinColumns.NEW_LAST_MAX_VOLUME.value],
                    'oScr', row[BinColumns.OLD_BASE_SCORE.value],
                    'nScr', row[BinColumns.NEW_BASE_SCORE.value]))


In [19]:
base_currency_symbol = 'USDT'
# base_currency_symbol = 'BTC'

# target_currency_symbol = 'BTC'
target_currency_symbol = 'ETH'
# target_currency_symbol = 'XMR'

currency_pair = '{}_{}'.format(base_currency_symbol, target_currency_symbol)
print('currency_pair={}'.format(currency_pair))

now = time.time()
now_int = int(now)

period_size_in_sec = 300
one_week_in_periods = 60*60*24*7 / period_size_in_sec

# old = historical data, i.e. can be analyzed together
# new = coming data, i.e. only the current period, previous periods and "old" periods are known, NOT future
# periods.
old_weeks = 6
new_weeks = 2
old_oldest_period_offset = old_weeks * one_week_in_periods
old_period_count = (old_weeks - new_weeks) * one_week_in_periods
new_oldest_period_offset = new_weeks * one_week_in_periods
new_period_count = new_weeks * one_week_in_periods

# NOTE: old_periods and new_periods intersect probably in one second.
old_periods_json = fetch_prices_json(currency_pair, now_int, old_oldest_period_offset, old_period_count,
                                     period_size_in_sec)
new_periods_json = fetch_prices_json(currency_pair, now_int, new_oldest_period_offset, new_period_count,
                                     period_size_in_sec)

currency_pair=USDT_ETH


https://poloniex.com/public?command=returnChartData&start=1501868974.0&currencyPair=USDT_ETH&period=300&end=1504288174.0


https://poloniex.com/public?command=returnChartData&start=1504288174.0&currencyPair=USDT_ETH&period=300&end=1505497774.0


In [20]:
now = time.time()

trading_fee_decimal = 0.0025

max_old_price_bin_count = 200
(old_periods, old_price_bin_count, old_max_price) = init_old_periods(old_periods_json, max_old_price_bin_count)

price_bins = init_price_bins(old_price_bin_count)

median_volume = old_periods[PeriodColumns.VOLUME.value].median()
mean_volume = old_periods[PeriodColumns.VOLUME.value].mean()
max_volume = old_periods[PeriodColumns.VOLUME.value].max()
multiplication_threshold = 2 * mean_volume / median_volume
min_volume_per_period_threshold = mean_volume
print('{}={}, {}={}'.format('multiplication_threshold', multiplication_threshold,
                            'min_volume_per_period_threshold', min_volume_per_period_threshold))
moving_overall_bin_max_volume = 0
calculate_old_bins_base_score(old_periods, multiplication_threshold, min_volume_per_period_threshold)

new_periods = init_periods(new_periods_json, old_period_count)

exchange_platform = ExchangePlatform(base_currency_symbol, target_currency_symbol)
exchange_platform.deposit(100)
calculate_new_bins_base_score_and_invest(new_periods,
                                         multiplication_threshold,
                                         min_volume_per_period_threshold,
                                         exchange_platform,
                                         old_price_bin_count,
                                         old_max_price,
                                         old_periods)


old_price_bin_count=88
multiplication_threshold=3.8932183063430768, min_volume_per_period_threshold=57311.04074568851


In [21]:
draw_graph(old_periods, new_periods, old_period_count, exchange_platform)

print('{}={:9.0f}'.format('moving_overall_bin_max_volume', moving_overall_bin_max_volume))
print_bins_result_debug_info(price_bins)

current_price = new_periods[PeriodColumns.CLOSE.value].iloc[-1]
print(exchange_platform.to_string(current_price))

print('\n{}={}'.format('elapsed_time', int(time.time()) - now))


AttributeError: 'OutstandingSellOrder' object has no attribute 'executed_on_period_index'